In [1]:
import json
import torch

In [3]:
from MyDataset import MyDataset
from torch.utils.data import DataLoader

In [ ]:
# lama_embedding_path = '../llama_embedding/tumor_3.1_finetuned/dataset_tensor/'
lama_embedding_path = '../llama_embedding/tumor_3.1_finetuned/dataset_tensor/'
bert_embedding_path = '../bert_embedding/tumor/dataset_tensor/'
roberta_embedding_path = '../roberta_embedding/tumor/dataset_tensor/'

In [5]:
train_data = MyDataset('train', lama_embedding_path, bert_embedding_path, roberta_embedding_path)
test_data =MyDataset('test', lama_embedding_path, bert_embedding_path, roberta_embedding_path)

In [6]:
train_data = DataLoader(train_data, batch_size=1, shuffle=False)
test_data = DataLoader(test_data, batch_size=1, shuffle=False) 

In [7]:
for i, data in enumerate(train_data):
    print(data)
    break

[tensor([[[ 0.6728, -1.4694, -0.2174,  ...,  1.9978,  0.8338, -1.1240]]]), tensor([[-0.9999, -0.9987,  1.0000,  ..., -0.9999,  0.9925, -0.9919]]), tensor([[-0.2379, -0.1287, -0.0410,  ...,  0.2268, -0.2190,  0.3663]]), tensor([3])]


In [8]:
l_rep, b_rep, r_rep, label = data

In [9]:
l_rep.squeeze().shape, b_rep.squeeze().shape, r_rep.squeeze().shape, label.item()

(torch.Size([4096]), torch.Size([1024]), torch.Size([1024]), 3)

In [10]:
# reps 별 코사인 유사도 평균 구해보기
def get_l1_distance(x1, x2):
    return ((x1 - x2).abs()).sum()
def get_l2_distance(x1, x2):
    return ((x1 - x2)**2).sum()**.5
def get_cosine_similarity(x1, x2):
    return (x1 * x2).sum() / ((x1**2).sum()**.5 * (x2**2).sum()**.5)

In [11]:
#get_cosine_similarity(reps_dict[0]['lama'][0].flatten() , reps_dict[0]['lama'][1].flatten()).item()

# Label 끼리의 코사인 유사도
## Train Data

In [24]:
reps_dict = {i : {'lama' : [],
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in train_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'].append(l_rep)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)
cnt == len(train_data)

True

In [25]:
all_dict = {i :{'lama' : [] ,'bert': [],'roberta': []} for i in range(6)}
for idx in range(6):
    # lama, bert, roberta 각각의 평균 코사인 유사도 구하기
    lama_cosine = []
    bert_cosine = []
    roberta_cosine = []
    for i in range(len(reps_dict[idx]['lama'])):
        for j in range(i+1, len(reps_dict[idx]['lama'])):
            lama_cosine.append(get_cosine_similarity(reps_dict[idx]['lama'][i].flatten() , reps_dict[idx]['lama'][j].flatten()).item())
            bert_cosine.append(get_cosine_similarity(reps_dict[idx]['bert'][i].flatten() , reps_dict[idx]['bert'][j].flatten()).item())
            roberta_cosine.append(get_cosine_similarity(reps_dict[idx]['roberta'][i].flatten() , reps_dict[idx]['roberta'][j].flatten()).item())

    all_dict[idx]['lama'] = torch.tensor(lama_cosine)
    all_dict[idx]['bert'] = torch.tensor(bert_cosine)
    all_dict[idx]['roberta'] = torch.tensor(roberta_cosine)

In [26]:
for i in range(6):
    print(f'Label {i} :')
    print(f'Lama mean : {all_dict[i]["lama"].mean()}, std : {all_dict[i]["lama"].std()}, min : {all_dict[i]["lama"].min()}, max : {all_dict[i]["lama"].max()}')
    print(f'Bert mean : {all_dict[i]["bert"].mean()}, std : {all_dict[i]["bert"].std()}, min : {all_dict[i]["bert"].min()}, max : {all_dict[i]["bert"].max()}')
    print(f'Roberta mean : {all_dict[i]["roberta"].mean()}, std : {all_dict[i]["roberta"].std()}, min : {all_dict[i]["roberta"].min()}, max : {all_dict[i]["roberta"].max()}')
    print()

Label 0 :
Lama mean : 0.7878373265266418, std : 0.07275228202342987, min : 0.49943384528160095, max : 0.9956824779510498
Bert mean : 0.9868912696838379, std : 0.01531849056482315, min : 0.9046784043312073, max : 0.9999618530273438
Roberta mean : 0.995334804058075, std : 0.002786097815260291, min : 0.9804888963699341, max : 0.9998929500579834

Label 1 :
Lama mean : 0.8334972262382507, std : 0.05799614265561104, min : 0.6394962668418884, max : 0.9911240935325623
Bert mean : 0.9860884547233582, std : 0.019397519528865814, min : 0.8408688902854919, max : 0.9999794960021973
Roberta mean : 0.9947881102561951, std : 0.0032459155190736055, min : 0.9727316498756409, max : 0.9999338984489441

Label 2 :
Lama mean : 0.8235480785369873, std : 0.06319207698106766, min : 0.6173321604728699, max : 0.9907570481300354
Bert mean : 0.9774080514907837, std : 0.026064366102218628, min : 0.8276188969612122, max : 1.0000001192092896
Roberta mean : 0.9951775670051575, std : 0.0029374866280704737, min : 0.97486

In [29]:
# 위 결과 excel로 저장
import pandas as pd
df = pd.DataFrame(all_dict)




TypeError: Could not convert [tensor([2.6525, 2.7510, 2.8855,  ..., 2.8181, 2.7641, 2.7405])
 tensor([2.7395, 2.8525, 2.8339,  ..., 2.8508, 2.8607, 2.8464])
 tensor([2.6740, 2.7766, 2.7359,  ..., 2.6389, 2.6475, 2.8617])
 tensor([2.7387, 2.8126, 2.8047,  ..., 2.8480, 2.8316, 2.8782])
 tensor([2.7060, 2.7691, 2.8304,  ..., 2.7397, 2.8065, 2.7115])
 tensor([2.8093, 2.8380, 2.8372,  ..., 2.8856, 2.8254, 2.8198])] to numeric

,0,1,2,3,4,5
lama,"[tensor(0.6625), tensor(0.7649), tensor(0.8938...","[tensor(0.7471), tensor(0.8704), tensor(0.8468...","[tensor(0.7140), tensor(0.7987), tensor(0.7736...","[tensor(0.7512), tensor(0.8205), tensor(0.8128...","[tensor(0.7697), tensor(0.8288), tensor(0.8944...","[tensor(0.8866), tensor(0.8769), tensor(0.8424..."
bert,"[tensor(0.9969), tensor(0.9901), tensor(0.9960...","[tensor(0.9965), tensor(0.9836), tensor(0.9897...","[tensor(0.9661), tensor(0.9824), tensor(0.9665...","[tensor(0.9965), tensor(0.9983), tensor(0.9969...","[tensor(0.9398), tensor(0.9459), tensor(0.9391...","[tensor(0.9249), tensor(0.9636), tensor(0.9971..."
roberta,"[tensor(0.9931), tensor(0.9960), tensor(0.9957...","[tensor(0.9960), tensor(0.9985), tensor(0.9973...","[tensor(0.9939), tensor(0.9955), tensor(0.9958...","[tensor(0.9910), tensor(0.9938), tensor(0.9950...","[tensor(0.9965), tensor(0.9944), tensor(0.9969...","[tensor(0.9977), tensor(0.9974), tensor(0.9977..."


## Test Data

In [15]:
reps_dict = {i : {'lama' : [],
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in test_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'].append(l_rep)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)
cnt == len(test_data)

True

In [16]:
all_dict = {i :{'lama' : [] ,'bert': [],'roberta': []} for i in range(6)}
for idx in range(6):
    # lama, bert, roberta 각각의 평균 코사인 유사도 구하기
    lama_cosine = []
    bert_cosine = []
    roberta_cosine = []
    for i in range(len(reps_dict[idx]['lama'])):
        for j in range(i+1, len(reps_dict[idx]['lama'])):
            lama_cosine.append(get_cosine_similarity(reps_dict[idx]['lama'][i].flatten() , reps_dict[idx]['lama'][j].flatten()).item())
            bert_cosine.append(get_cosine_similarity(reps_dict[idx]['bert'][i].flatten() , reps_dict[idx]['bert'][j].flatten()).item())
            roberta_cosine.append(get_cosine_similarity(reps_dict[idx]['roberta'][i].flatten() , reps_dict[idx]['roberta'][j].flatten()).item())

    all_dict[idx]['lama'] = torch.tensor(lama_cosine)
    all_dict[idx]['bert'] = torch.tensor(bert_cosine)
    all_dict[idx]['roberta'] = torch.tensor(roberta_cosine)

In [17]:
print('--------------Test Data-------------------')
for i in range(6):
    print(f'Label {i} :')
    print(f'Lama mean : {all_dict[i]["lama"].mean()}, std : {all_dict[i]["lama"].std()}, min : {all_dict[i]["lama"].min()}, max : {all_dict[i]["lama"].max()}')
    print(f'Bert mean : {all_dict[i]["bert"].mean()}, std : {all_dict[i]["bert"].std()}, min : {all_dict[i]["bert"].min()}, max : {all_dict[i]["bert"].max()}')
    print(f'Roberta mean : {all_dict[i]["roberta"].mean()}, std : {all_dict[i]["roberta"].std()}, min : {all_dict[i]["roberta"].min()}, max : {all_dict[i]["roberta"].max()}')
    print()

--------------Test Data-------------------
Label 0 :
Lama mean : 0.779055655002594, std : 0.07247831672430038, min : 0.5458939075469971, max : 0.9934595227241516
Bert mean : 0.9793155193328857, std : 0.023359069600701332, min : 0.8788700699806213, max : 0.999977707862854
Roberta mean : 0.994115948677063, std : 0.0030253378208726645, min : 0.9835890531539917, max : 0.9998114705085754

Label 1 :
Lama mean : 0.7998738884925842, std : 0.07061295211315155, min : 0.6148551106452942, max : 0.989418089389801
Bert mean : 0.9911202788352966, std : 0.010052916593849659, min : 0.9487963914871216, max : 0.9999092817306519
Roberta mean : 0.9940482974052429, std : 0.004317734390497208, min : 0.9637802839279175, max : 0.9999279975891113

Label 2 :
Lama mean : 0.812993049621582, std : 0.060396865010261536, min : 0.6495745778083801, max : 0.9872453808784485
Bert mean : 0.9762211441993713, std : 0.02486557327210903, min : 0.8885114192962646, max : 0.999900758266449
Roberta mean : 0.9934955835342407, std 

# 다른 라벨들과의 코사인 유사도 비교
## TrainData

In [18]:
reps_dict = {i : {'lama' : [],
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in train_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'].append(l_rep)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)
cnt == len(train_data)

True

In [19]:
all_dict = {i : {j :{'lama' : [] ,'bert': [],'roberta': []}  for j in range(6) if i != j } for i in range(6)}
for i in range(6):
    # lama, bert, roberta 각각의 평균 코사인 유사도 구하기

    for j in range(6):
        lama_cosine = []
        bert_cosine = []
        roberta_cosine = []
        if i != j :
            for l in range(len(reps_dict[i]['lama'])):
                for m in range(len(reps_dict[j]['lama'])):
                    lama_cosine.append(get_cosine_similarity(reps_dict[i]['lama'][l].flatten() , reps_dict[j]['lama'][m].flatten()).item())
                    bert_cosine.append(get_cosine_similarity(reps_dict[i]['bert'][l].flatten() , reps_dict[j]['bert'][m].flatten()).item())
                    roberta_cosine.append(get_cosine_similarity(reps_dict[i]['roberta'][l].flatten() , reps_dict[j]['roberta'][m].flatten()).item())
            
            all_dict[i][j]['lama'] = torch.tensor(lama_cosine)
            all_dict[i][j]['bert'] = torch.tensor(bert_cosine)
            all_dict[i][j]['roberta'] = torch.tensor(roberta_cosine)
        else:
            continue

In [20]:
# i, j 레이블 간의 평균 코사인 유사도 구하기
for i in range(6):
    for j in range(6):
        if i != j:
            print(f'Label {i} - Label {j} :')
            print(f'Lama mean : {all_dict[i][j]["lama"].mean()}, std : {all_dict[i][j]["lama"].std()}, min : {all_dict[i][j]["lama"].min()}, max : {all_dict[i][j]["lama"].max()}')
            print(f'Bert mean : {all_dict[i][j]["bert"].mean()}, std : {all_dict[i][j]["bert"].std()}, min : {all_dict[i][j]["bert"].min()}, max : {all_dict[i][j]["bert"].max()}')
            print(f'Roberta mean : {all_dict[i][j]["roberta"].mean()}, std : {all_dict[i][j]["roberta"].std()}, min : {all_dict[i][j]["roberta"].min()}, max : {all_dict[i][j]["roberta"].max()}')
            print()

Label 0 - Label 1 :
Lama mean : 0.6684419512748718, std : 0.04753425717353821, min : 0.5020684599876404, max : 0.8779376745223999
Bert mean : 0.9863342642784119, std : 0.01797829382121563, min : 0.8594829440116882, max : 0.9999076724052429
Roberta mean : 0.9947792291641235, std : 0.0030403973069041967, min : 0.9745878577232361, max : 0.9994104504585266

Label 0 - Label 2 :
Lama mean : 0.6305508613586426, std : 0.05471551790833473, min : 0.4781194031238556, max : 0.8777139782905579
Bert mean : 0.9821799397468567, std : 0.020543904975056648, min : 0.8548378348350525, max : 0.9998860359191895
Roberta mean : 0.9945757389068604, std : 0.003455876838415861, min : 0.970973014831543, max : 0.9991005659103394

Label 0 - Label 3 :
Lama mean : 0.727867066860199, std : 0.062253836542367935, min : 0.47244030237197876, max : 0.8987920880317688
Bert mean : 0.9859519004821777, std : 0.016293762251734734, min : 0.8842725157737732, max : 0.9999192953109741
Roberta mean : 0.9951332211494446, std : 0.0028

## Test  Data

In [21]:
reps_dict = {i : {'lama' : [],
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in test_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'].append(l_rep)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)
cnt == len(test_data)

True

In [22]:
all_dict = {i : {j :{'lama' : [] ,'bert': [],'roberta': []}  for j in range(6) if i != j } for i in range(6)}
for i in range(6):
    # lama, bert, roberta 각각의 평균 코사인 유사도 구하기

    for j in range(6):
        lama_cosine = []
        bert_cosine = []
        roberta_cosine = []
        if i != j :
            for l in range(len(reps_dict[i]['lama'])):
                for m in range(len(reps_dict[j]['lama'])):
                    lama_cosine.append(get_cosine_similarity(reps_dict[i]['lama'][l].flatten() , reps_dict[j]['lama'][m].flatten()).item())
                    bert_cosine.append(get_cosine_similarity(reps_dict[i]['bert'][l].flatten() , reps_dict[j]['bert'][m].flatten()).item())
                    roberta_cosine.append(get_cosine_similarity(reps_dict[i]['roberta'][l].flatten() , reps_dict[j]['roberta'][m].flatten()).item())
            
            all_dict[i][j]['lama'] = torch.tensor(lama_cosine)
            all_dict[i][j]['bert'] = torch.tensor(bert_cosine)
            all_dict[i][j]['roberta'] = torch.tensor(roberta_cosine)
        else:
            continue

In [23]:
# i, j 레이블 간의 평균 코사인 유사도 구하기
for i in range(6):
    for j in range(6):
        if i != j:
            print(f'Label {i} - Label {j} :')
            print(f'Lama mean : {all_dict[i][j]["lama"].mean()}, std : {all_dict[i][j]["lama"].std()}, min : {all_dict[i][j]["lama"].min()}, max : {all_dict[i][j]["lama"].max()}')
            print(f'Bert mean : {all_dict[i][j]["bert"].mean()}, std : {all_dict[i][j]["bert"].std()}, min : {all_dict[i][j]["bert"].min()}, max : {all_dict[i][j]["bert"].max()}')
            print(f'Roberta mean : {all_dict[i][j]["roberta"].mean()}, std : {all_dict[i][j]["roberta"].std()}, min : {all_dict[i][j]["roberta"].min()}, max : {all_dict[i][j]["roberta"].max()}')
            print()

Label 0 - Label 1 :
Lama mean : 0.6727877855300903, std : 0.05061781033873558, min : 0.5362988710403442, max : 0.8274257183074951
Bert mean : 0.9854336977005005, std : 0.01720646768808365, min : 0.9064790606498718, max : 0.9998049736022949
Roberta mean : 0.9939550757408142, std : 0.003750670701265335, min : 0.9695395827293396, max : 0.9986449480056763

Label 0 - Label 2 :
Lama mean : 0.6256682872772217, std : 0.04690054431557655, min : 0.5026456713676453, max : 0.7708511352539062
Bert mean : 0.9785114526748657, std : 0.023559190332889557, min : 0.8749093413352966, max : 0.999830424785614
Roberta mean : 0.9937593936920166, std : 0.0033980885054916143, min : 0.9778833389282227, max : 0.9988173842430115

Label 0 - Label 3 :
Lama mean : 0.7160433530807495, std : 0.06317172944545746, min : 0.48789769411087036, max : 0.8927400708198547
Bert mean : 0.9812770485877991, std : 0.021685641258955002, min : 0.8784774541854858, max : 0.9998173117637634
Roberta mean : 0.9943814873695374, std : 0.0028

# 다른 라벨들과의 코사인 유사도

In [48]:
reps_dict = {i : {'lama' : [],
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in train_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'].append(l_rep)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)
cnt == len(train_data)

True

In [49]:
all_dict = {i : {j :{'lama' : [] ,'bert': [],'roberta': []}  for j in range(6) if i != j } for i in range(6)}
for i in range(6):
    # lama, bert, roberta 각각의 평균 코사인 유사도 구하기

    for j in range(6):
        lama_cosine = []
        bert_cosine = []
        roberta_cosine = []
        if i != j :
            for l in range(len(reps_dict[i]['lama'])):
                for m in range(len(reps_dict[j]['lama'])):
                    lama_cosine.append(get_cosine_similarity(reps_dict[i]['lama'][l].flatten() , reps_dict[j]['lama'][m].flatten()).item())
                    bert_cosine.append(get_cosine_similarity(reps_dict[i]['bert'][l].flatten() , reps_dict[j]['bert'][m].flatten()).item())
                    roberta_cosine.append(get_cosine_similarity(reps_dict[i]['roberta'][l].flatten() , reps_dict[j]['roberta'][m].flatten()).item())
            
            all_dict[i][j]['lama'] = torch.tensor(lama_cosine)
            all_dict[i][j]['bert'] = torch.tensor(bert_cosine)
            all_dict[i][j]['roberta'] = torch.tensor(roberta_cosine)
        else:
            continue

In [50]:
# i, j 레이블 간의 평균 코사인 유사도 구하기
for i in range(6):
    for j in range(6):
        if i != j:
            print(f'Label {i} - Label {j} :')
            print(f'Lama mean : {all_dict[i][j]["lama"].mean()}, std : {all_dict[i][j]["lama"].std()}, min : {all_dict[i][j]["lama"].min()}, max : {all_dict[i][j]["lama"].max()}')
            print(f'Bert mean : {all_dict[i][j]["bert"].mean()}, std : {all_dict[i][j]["bert"].std()}, min : {all_dict[i][j]["bert"].min()}, max : {all_dict[i][j]["bert"].max()}')
            print(f'Roberta mean : {all_dict[i][j]["roberta"].mean()}, std : {all_dict[i][j]["roberta"].std()}, min : {all_dict[i][j]["roberta"].min()}, max : {all_dict[i][j]["roberta"].max()}')
            print()

Label 0 - Label 1 :
Lama mean : 0.6684419512748718, std : 0.04753425717353821, min : 0.5020684599876404, max : 0.8779376745223999
Bert mean : 0.9863342642784119, std : 0.01797829382121563, min : 0.8594829440116882, max : 0.9999076724052429
Roberta mean : 0.9947792291641235, std : 0.0030403973069041967, min : 0.9745878577232361, max : 0.9994104504585266

Label 0 - Label 2 :
Lama mean : 0.6305508613586426, std : 0.05471551790833473, min : 0.4781194031238556, max : 0.8777139782905579
Bert mean : 0.9821799397468567, std : 0.020543904975056648, min : 0.8548378348350525, max : 0.9998860359191895
Roberta mean : 0.9945757389068604, std : 0.003455876838415861, min : 0.970973014831543, max : 0.9991005659103394

Label 0 - Label 3 :
Lama mean : 0.727867066860199, std : 0.062253836542367935, min : 0.47244030237197876, max : 0.8987920880317688
Bert mean : 0.9859519004821777, std : 0.016293762251734734, min : 0.8842725157737732, max : 0.9999192953109741
Roberta mean : 0.9951332211494446, std : 0.0028

# Test

In [51]:
reps_dict = {i : {'lama' : [],
                  'bert' : [],
                'roberta' : [] } for i in range(6)}
cnt = 0
for data in test_data:
    cnt += 1
    l_rep, b_rep, r_rep, label = data
    l_rep = l_rep.squeeze()
    b_rep = b_rep.squeeze()
    r_rep = r_rep.squeeze()
    label = label.item()
    reps_dict[label]['lama'].append(l_rep)
    reps_dict[label]['bert'].append(b_rep)
    reps_dict[label]['roberta'].append(r_rep)
cnt == len(test_data)

True

In [52]:
all_dict = {i : {j :{'lama' : [] ,'bert': [],'roberta': []}  for j in range(6) if i != j } for i in range(6)}
for i in range(6):
    # lama, bert, roberta 각각의 평균 코사인 유사도 구하기

    for j in range(6):
        lama_cosine = []
        bert_cosine = []
        roberta_cosine = []
        if i != j :
            for l in range(len(reps_dict[i]['lama'])):
                for m in range(len(reps_dict[j]['lama'])):
                    lama_cosine.append(get_cosine_similarity(reps_dict[i]['lama'][l].flatten() , reps_dict[j]['lama'][m].flatten()).item())
                    bert_cosine.append(get_cosine_similarity(reps_dict[i]['bert'][l].flatten() , reps_dict[j]['bert'][m].flatten()).item())
                    roberta_cosine.append(get_cosine_similarity(reps_dict[i]['roberta'][l].flatten() , reps_dict[j]['roberta'][m].flatten()).item())
            
            all_dict[i][j]['lama'] = torch.tensor(lama_cosine)
            all_dict[i][j]['bert'] = torch.tensor(bert_cosine)
            all_dict[i][j]['roberta'] = torch.tensor(roberta_cosine)
        else:
            continue

In [53]:
# i, j 레이블 간의 평균 코사인 유사도 구하기
for i in range(6):
    for j in range(6):
        if i != j:
            print(f'Label {i} - Label {j} :')
            print(f'Lama mean : {all_dict[i][j]["lama"].mean()}, std : {all_dict[i][j]["lama"].std()}, min : {all_dict[i][j]["lama"].min()}, max : {all_dict[i][j]["lama"].max()}')
            print(f'Bert mean : {all_dict[i][j]["bert"].mean()}, std : {all_dict[i][j]["bert"].std()}, min : {all_dict[i][j]["bert"].min()}, max : {all_dict[i][j]["bert"].max()}')
            print(f'Roberta mean : {all_dict[i][j]["roberta"].mean()}, std : {all_dict[i][j]["roberta"].std()}, min : {all_dict[i][j]["roberta"].min()}, max : {all_dict[i][j]["roberta"].max()}')
            print()

Label 0 - Label 1 :
Lama mean : 0.6727877855300903, std : 0.05061781033873558, min : 0.5362988710403442, max : 0.8274257183074951
Bert mean : 0.9854336977005005, std : 0.01720646768808365, min : 0.9064790606498718, max : 0.9998049736022949
Roberta mean : 0.9939550757408142, std : 0.003750670701265335, min : 0.9695395827293396, max : 0.9986449480056763

Label 0 - Label 2 :
Lama mean : 0.6256682872772217, std : 0.04690054431557655, min : 0.5026456713676453, max : 0.7708511352539062
Bert mean : 0.9785114526748657, std : 0.023559190332889557, min : 0.8749093413352966, max : 0.999830424785614
Roberta mean : 0.9937593936920166, std : 0.0033980885054916143, min : 0.9778833389282227, max : 0.9988173842430115

Label 0 - Label 3 :
Lama mean : 0.7160433530807495, std : 0.06317172944545746, min : 0.48789769411087036, max : 0.8927400708198547
Bert mean : 0.9812770485877991, std : 0.021685641258955002, min : 0.8784774541854858, max : 0.9998173117637634
Roberta mean : 0.9943814873695374, std : 0.0028